# Multiple Variable Linear Regression

## Goals

- Extend our regression model routines to support multiple features
- Extend data structures to support multiple features
- Rewrite prediction, Python functions to compute cost function and its gradient to support multiple features
- Utilize NumPy function `numpy.dot()` or the ndarray object method `ndarray.dot()` to vectorize the implementation for speed and simplicity

## Tools

In this lab, we will make use of
- NumPy array, a popular library for scientific computing
- Matplotlib, a popular library for plotting data

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Notations

The linear regression model is formulated as
$$f_{w, b}(x) = w x + b$$

Following is a summary of notation you will encounter

General notation | Description | Python (if applicable 
-----------------| ------------| --------------------- 
$a$ | scalar, non-bold |
$\mathbf{a}$ | vector, bold |
$\mathbf{x}$ | training examlpe feature values  | `x_train`
$\mathbf{y}$ | training example targets         | `y_train`
$x^{(i)}, y^{(i)}$    | $i^\mathrm{th}$ training example | `x_i`, `y_i`
$m$          | number of training examples      | `m`
$$\mathbf{w}= (w_{1}, \ldots, w_{n})$$ | parameter: weight | `w`
$b$          | parameter: bias   | `b`
$f_{\mathbf{w},b}(x^{(i)})$ | The result of the model evaluation at $x^{(i)}$ parameterized by the weight $\mathbf{w}$ and the bias $b$ | `f_wb`

# 2 Problem statement

You will use the motivating example of house price prediction. The training dataset contains three examples with four features (size, bedrooms, floors and age) shown in the table below. Note that we use square foot instead of square meter for the area. This causes an issue, which we will try to resolve in the next *self-study notebook*.

| Size (sqft) | Number of Bedrooms  | Number of floors | Age of  Home | Price (1000 £)  |   
| ----------------| ------------------- |----------------- |--------------|-------------- |  
| 600            | 2                   | 1                | 45           | 150           |  
| 480            | 1                   | 1                | 40           | 110           |  
| 750             | 3                   | 2                | 35           | 190           | 

You will build a linear regression model using these values so you can then predict the price for other houses. For example, a house with 550 sqft, 2 bedrooms, 1 floor and 37 years old.

In [2]:
X_train = np.array([[600, 2, 1, 45], [480, 1, 1, 40], [750, 3, 2, 35]])
y_train = np.array([150, 110, 190])

## 2.0 Solution using scikit-learn

Before doing anything, let us try to obtain the parameter solution using scikit-learn. The results will be used as reference parameters for comparison in the following implementation.

In [3]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
w_ref = linear_model.coef_
b_ref = linear_model.intercept_
print(f"w_ref = {w_ref}")
print(f"b_ref = {b_ref}")

linear_model.predict(X_train)

w_ref = [ 0.30769626  0.01179659 -0.03692984  0.61293052]
b_ref = -62.186290858870166


array([150., 110., 190.])

## 2.1 Matrix X containing our examples
Similar to the table above, examples are stored in a NumPy matrix `X_train`. Each row of the matrix represents one example. When you have $m$ training examples ($m = 3$ in our example), and there are $n$ features ($n = 4$ our example), $\mathbf{X}$ is a matrix with dimensions ($m$, $n$) (m rows, n columns).


$$\mathbf{X} = 
\begin{pmatrix}
 x^{(0)}_0 & x^{(0)}_1 & \cdots & x^{(0)}_{n-1} \\ 
 x^{(1)}_0 & x^{(1)}_1 & \cdots & x^{(1)}_{n-1} \\
 \cdots \\
 x^{(m-1)}_0 & x^{(m-1)}_1 & \cdots & x^{(m-1)}_{n-1} 
\end{pmatrix}
$$
notation:
- $\mathbf{x}^{(i)}$ is vector containing example $i$. $\mathbf{x}^{(i)}$ $ = (x^{(i)}_0, x^{(i)}_1, \cdots,x^{(i)}_{n-1})$
- $x^{(i)}_j$ is element $j$ in example $i$. The superscript in parenthesis indicates the example number while the subscript represents an element.  

Display the input data.

In [4]:
# data is stored in numpy array (matrix)
print("X =", repr(X_train))
(m, n) = X_train.shape  # well, we don't even need to store this info into m, and n -- it is obvious.
print("y =", repr(y_train))

X = array([[600,   2,   1,  45],
       [480,   1,   1,  40],
       [750,   3,   2,  35]])
y = array([150, 110, 190])


## 2.2 Parameter vector w, b

* $\mathbf{w}$ is a vector with $n$ elements.
  - Each element contains the parameter associated with one feature.
  - in our dataset, n is 4.
  - notionally, we draw this as a column vector

$$\mathbf{w} = \begin{bmatrix}
w_0 \\ 
w_1 \\
\cdots\\
w_{n-1}
\end{bmatrix}
$$
* $b$ is a scalar parameter.

For demonstration, $\mathbf{w}$ and $b$ will be loaded with some initial selected values that are near the optimal. $\mathbf{w}$ is a 1-D NumPy vector.

In [5]:
b_init = b_ref
w_init = w_ref

print(f"w_init shape: {w_init.shape}")
print(f"b_init type: {type(b_init)}")

w_init shape: (4,)
b_init type: <class 'numpy.float64'>


# 3 Model Prediction With Multiple Variables
The model's prediction with multiple variables is given by the linear model:

$$ f_{\mathbf{w},b}(\mathbf{x}) =  w_0x_0 + w_1x_1 +... + w_{n-1}x_{n-1} + b \tag{1}$$
or in vector notation:
$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b  \tag{2} $$ 
where $\cdot$ is a vector `dot product`

To demonstrate the dot product, we will implement prediction using (1) and (2).

## 3.1 Compute single prediction (element by element) using loop

In [6]:
def predict_with_loop(x, w, b):
    n = x.shape[0]   # number of features
    s = 0
    for i in range(n):
        s += x[i] * w[i]
    s += b
    return s

In [7]:
# Make prediction for one training example
x_vec = X_train[0,:]    # take the first element in the training example.
# Note that x_vec is 1D numpy array
print(f"x_vec shape: {x_vec.shape}")

f_wb = predict_with_loop(x_vec, w_init, b_init)
print(f"prediction #0 = {f_wb}")
# Of course we can also write
f_wb = predict_with_loop(X_train[1,:], w_init, b_init)
print(f"prediction #1 = {f_wb}")

x_vec shape: (4,)
prediction #0 = 150.0
prediction #1 = 109.99999999999994


## 3.2 Prediction using `numpy.dot()`

The above implementation is inefficient in terms of running time and usage convenience. We can make use of vector operations to speed up predictions. Note that the sum given by
$$s = \sum\limits_{j=1}^{n}x_{j}^{(i)} x_{j}^{(i)} w_j$$
for one training example is nothing else but the dot product between two vectors 
$$
\mathbf{x}^{(i)} = (x_{1}^{(i)},\ldots x_{n}^{(i)}), \quad \text{and} \quad \mathbf{w} = (w_1, \ldots, w_n).
$$
That is, we can write
$$
f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} = \mathbf{x}^{(i)} \cdot \mathbf{w}
$$

Thus, we can rewrite the above Python function with a much short piece of code. It is so short that it does not even make sense to write down the function. But we will do it anyway.

In [8]:
def predict(x, w, b):
    return np.dot(x, w) + b

In [9]:
# Let us repeat computing predictions as above (but now for all three training examples)
m = X_train.shape[0]
for i in range(m):
    f_wb = predict(X_train[i,:], w_init, b_init)
    print(f"prediction #{i} = {f_wb}")

prediction #0 = 150.0
prediction #1 = 109.99999999999997
prediction #2 = 189.99999999999997


**Power of NumPy array**

The results and shapes are the same as the previous version which used looping. Going forward, `np.dot` will be used for these operations. The prediction is now a single statement. Most routines will implement it directly rather than calling a separate predict routine.

What is even more interesting is that the above line of code `np.dot(x, w) + b` work not only for one example after one example. It actually works for a list of training example. What you need to do is simple. Instead of using one particular example with `X_train[i,:]`, we can use the whole matrix `X_train`.

In [10]:
print(predict(X_train, w_init, b_init))  # you can see they give the same results as above

[150. 110. 190.]


The explanation for this is simple. Each training example is represented by a row in the matrix $\mathbf{X}$. Thus, the dot product $\mathbf{x}^{(i)} \cdot \mathbf{w}$ is actually the dot product of the row $i^\mathrm{th}$ in the matrix $\mathbf{X}$ and the parameter vector $\mathbf{w}$. The matrix multiplication $\mathbf{X}\cdot \mathbf{w}$ is nothing but the dot products of all of the row vectors in $\mathbf{X}$ and $\mathbf{w}$. Thus, the dot products between multiple training examples $\mathbf{x}^{(i)}$ and one vector $\mathbf{w}$ can be effectively combined into one dot product between the whole 2D array $\mathbf{X}$ and the vector $\mathbf{w}$.

\begin{equation*}
\mathbf{X}\cdot \mathbf{w} =\begin{pmatrix}
 x^{(0)}_0 & x^{(0)}_1 & \cdots & x^{(0)}_{n-1} \\ 
 x^{(1)}_0 & x^{(1)}_1 & \cdots & x^{(1)}_{n-1} \\
 \vdots & \ddots & \ddots & \vdots\\
 x^{(m-1)}_0 & x^{(m-1)}_1 & \cdots & x^{(m-1)}_{n-1} 
\end{pmatrix}\cdot \begin{pmatrix}
w_0 \\ 
w_1 \\
\vdots\\
w_{n-1}
\end{pmatrix} = \begin{pmatrix}
\mathbf{x}^{(0)} \\ \mathbf{x}^{(1)} \\ \vdots \\ \mathbf{x}^{(m-1)}
\end{pmatrix} \cdot \mathbf{w} 
= \begin{pmatrix}
\mathbf{x}^{(0)} \cdot \mathbf{w} \\
\mathbf{x}^{(1)} \cdot \mathbf{w} \\
\vdots\\
\mathbf{x}^{(m-1)} \cdot \mathbf{w}
\end{pmatrix}
\end{equation*}

If you are still unsure what's going on, the above dot product between 2D array and a vector is nothing but the matrix multiplication between $\mathbf{X}$ and a column vector given by rearrange the 1D "mathematical" vector $\mathbf{w}$. To do this, can make `w` into a 2D array with one column and $n$ rows using `w.reshape((-1, 1))`. The value `-1` means the number of rows is determined appropriately after using `1` column for the `reshape` operator. See the code below for better understanding.

In [11]:
w_column = w_init.reshape((-1, 1))
print("w_init =", w_init)
print("w_col =", w_column)
print(f"w_col shape: {w_column.shape}")
print("="*60)
print("X_train-dot-w_col =", X_train.dot(w_column))     # return a column vector as a 2D array
print("X_train-dot-w =", X_train.dot(w_init))           # return a 1D "mathematical" vector, same values

w_init = [ 0.30769626  0.01179659 -0.03692984  0.61293052]
w_col = [[ 0.30769626]
 [ 0.01179659]
 [-0.03692984]
 [ 0.61293052]]
w_col shape: (4, 1)
X_train-dot-w_col = [[212.18629086]
 [172.18629086]
 [252.18629086]]
X_train-dot-w = [212.18629086 172.18629086 252.18629086]


# 4 Compute Cost with Multiple variables

The equation for the cost function with multiple variables $J(\mathbf{w},b)$ is:
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{3}$$ 
where:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{4} $$ 


In contrast to previous labs, $\mathbf{w}$ and $\mathbf{x}^{(i)}$ are vectors rather than scalars supporting multiple features.

Below is an implementation of equations (3) and (4). Note that this uses a *standard pattern for this course* where a for loop over all `m` examples is used.

In [12]:
def compute_cost(X, y, w, b): 
    """
    compute cost
    Arguments:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """
    m = X.shape[0]
    f_wb = np.dot(X, w) + b
    cost = np.sum((f_wb - y) ** 2) / (2 * m)
    return cost

In [13]:
# Compute and display cost using the reference parameter
# solution obtained from scikit-learn
cost = compute_cost(X_train, y_train, w_ref, b_ref)
print(f"Cost at initial parameters = {cost}")

Cost at initial parameters = 6.731613057885968e-28


# 5 Gradient Descent with Multiple Variables

Gradient descent for multiple variables:

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline\;
& w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{5}  \; & \text{for j = 0..n-1}\newline
&b\ \ = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

where, n is the number of features, parameters $w_j$,  $b$, are updated simultaneously and where  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{6}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{7}
\end{align}
$$
* m is the number of training examples in the data set

    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ is the model's prediction, while $y^{(i)}$ is the target value

## 5.1 Compute Gradient with Multiple Variables

### 5.1.1. Option 1

An implementation for calculating the equations (6) and (7) is below. There are many ways to implement this. In this version, there is an
- outer loop over all m examples. 
    - $\frac{\partial J(\mathbf{w},b)}{\partial b}$ for the example can be computed directly and accumulated
    - in a second loop over all n features:
        - $\frac{\partial J(\mathbf{w},b)}{\partial w_j}$ is computed for each $w_j$.

In [14]:
def compute_gradient_with_two_loop(X, y, w, b): 
    """See the doc-string in the next function."""
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_dw, dj_db

As in the last Week, it is always wise to double check the implementation of gradient descent.

In [15]:
b_test = b_ref + 0.1
w_test = w_ref + 0.5
def test(func_grad):
    epsilon = 1e-4
    n = X_train.shape[1]
    dJ_dw_approx = []
    for i in range(n):
        w1 = w_test.copy(); w1[i] -= epsilon
        w2 = w_test.copy(); w2[i] += epsilon
        J1 = compute_cost(X_train, y_train, w1, b_test)
        J2 = compute_cost(X_train, y_train, w2, b_test)
        dJ_dw_approx.append((J2 - J1) / (2*epsilon))

    dJ_dw_fd = np.array(dJ_dw_approx)  # fd stands for finite difference
    
    # use specific function that is passed to test()
    dJ_dw, dJ_db = func_grad(X_train, y_train, w_test, b_test)  
    
    print(f"finite difference dJ_dw = {dJ_dw_fd}")
    print(f"                  dJ_dw = {dJ_dw}")

test(compute_gradient_with_two_loop)

finite difference dJ_dw = [205370.99999987    698.20000008    458.46666664  12952.33333338]
                  dJ_dw = [205371.            698.2           458.46666667  12952.33333333]


### 5.1.2 Option 2

It turns out that we can aliviate one loop in the implementation of computing the cost gradient. Indeed, the loop over all the features, namely `for j in range(n)` is not needed. We can see that the index `j` apprears on both sides of the expression
```Python
dj_dw[j] = dj_dw[j] + err * X[i, j]
```
This shows the sign that we can replace this line code and remove the `for` loop over the running index `j` by vectorization concept. Indeed, the whole loop
```Python
for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]
```
is equivalent to
```Python
dj_dw = dj_dw + err * X[i,:]
```

Keepind this in mind, we can rewrite the function `compute_gradient_with_loop` as follows.

In [16]:
def compute_gradient_with_single_loop(X, y, w, b): 
    """See the doc-string in the next function definition"""
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        dj_dw = dj_dw + err * X[i,:]   # no need to write for-loop
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_dw, dj_db

In [17]:
test(compute_gradient_with_single_loop)

finite difference dJ_dw = [205370.99999987    698.20000008    458.46666664  12952.33333338]
                  dJ_dw = [205371.            698.2           458.46666667  12952.33333333]


### 5.1.3 Option 3

It is probably natural to ask whether we can completely remove the last `for` loop. Answer is **yes**. Just think about the following code.

In [18]:
def compute_gradient(X, y, w, b):
    """Compute the gradient for linear regression 
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b.
    """
    m, n = X.shape
    f_wb = np.dot(X, w) + b
    err = f_wb - y
    dj_dw = np.dot(err, X) / m
    dj_db = np.sum(err) / m
    return dj_dw, dj_db

In [19]:
test(compute_gradient)

finite difference dJ_dw = [205370.99999987    698.20000008    458.46666664  12952.33333338]
                  dJ_dw = [205371.            698.2           458.46666667  12952.33333333]


## 5.2 Gradient Descent with Multiple Variables

In [20]:
import copy

In [21]:
def gradient_descent(X, y, w_in, b_in, func_value, func_grad, alpha, num_iters): 
    """
    Perform batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters  
      b_in (scalar)       : initial model parameter
      cost_function       : function to compute cost
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : Updated values of parameters 
      b (scalar)       : Updated value of parameter 
      """
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_dw, dj_db = func_grad(X, y, w, b)   ##None

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               ##None
        b = b - alpha * dj_db               ##None
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion
            J_history.append(func_value(X, y, w, b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% np.ceil(num_iters / 10) == 0:
            print("Iteration {0:4d}: Cost {1:8.2f}".format(i, J_history[-1]))
        
    return w, b, J_history #return final w,b and J history for graphing

In [30]:
# initialize parameters
initial_w = np.zeros_like(w_init)
initial_b = 0.0
# some gradient descent settings
iterations = 1000000
alpha = 5.0e-6

In [33]:
# run gradient descent 
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b, \
                                            compute_cost, compute_gradient, alpha, iterations)
print(f"\nb,w found by gradient descent:\n" + "="*60)
print(f"b_final = {b_final}")
print(f"w_final = {w_final}")
print("="*60)
print(f"w_ref = {w_ref}")
print(f"b_ref = {b_ref}")
print("="*60)
m,_ = X_train.shape
for i in range(m):
    print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.2f} -- target value: {y_train[i]}")

Iteration    0: Cost 10174.61
Iteration 100000: Cost     6.74
Iteration 200000: Cost     6.74
Iteration 300000: Cost     6.74
Iteration 400000: Cost     6.74
Iteration 500000: Cost     6.74
Iteration 600000: Cost     6.74
Iteration 700000: Cost     6.74
Iteration 800000: Cost     6.74
Iteration 900000: Cost     6.74

b,w found by gradient descent:
b_final = -0.9946625257155384
w_final = [ 0.2639255   3.2868849  -2.19745429 -0.33688748]
w_ref = [ 0.30769626  0.01179659 -0.03692984  0.61293052]
b_ref = -62.186290858870166
prediction: 146.58 -- target value: 150
prediction: 113.30 -- target value: 110
prediction: 190.62 -- target value: 190


In [34]:
linear_model.predict(X_train)

array([150., 110., 190.])

**Remark**

You can see that the result of the gradient descent and the Linear Regression model from sklearn do not agree with each other well. This is because the gradient descent has not converged well enough. We will learn that the problem is that the values of different features are in different range and this makes the gradient descent struggle to converge. To overcome this issue, we will learn about feature normalization in the next lecture to fix this issue.